# Proyecto 1 - Analítica de textos 
## Análisis de sentimientos en películas

### Integrantes:
- Daniel Aguilera
- Vanessa Martínez
- Cristian Sánchez

### Descripción del problema
El objetivo de este notebook es realizar un análisis sobre comentarios en películas, para el cual se tiene un dataset con 5000  comentarios en español. En primera instacia tenemos el comentario de la persona y una categorización de esta, si es positiva o negativa. Este proyecto se enfocará en crear un modelo que por medio de esos datos de entrenamiento pueda verificar si un comentario es positivo o negativo, usando analítica de textos.

Se tiene como resultado los siguientes objetivos:
- Aplicar la metodología de analítica de textos para la construcción de soluciones de
analítica alineadas con los objetivos del negocio en un contexto de aplicación.
- Interactuar con un grupo interdisciplinario para validar y mejorar la calidad de la
solución analítica planteada.

Por medio de 2 etapas este proyecto busca cumplir con los objetivos propuestos:
- **Etapa 1:** Contrucción de modelos de analítica de textos.
- **Etapa 2:** Automatización y uso de modelos de analítica de textos.

## Referencias
1. [Datos a utilizar: MovieReviews.csv](./datos/MovieReviews.csv)
2. [Diccionario de datos: DiccionarioPeliculas.xlsx](./datos/DiccionarioPeliculas.xlsx)
3. [Datos de prueba: MovieReviewsPruebas.csv](./datos/MovieReviewsPruebas.csv)
4. [Información del negocio](./datos/enunciado.pdf)



# ***Etapa 1***

## 1. Entendimiento de datos

### 1.1 Importación de librerías

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

### 1.2 Carga de datos
En este apartado se realiza la carga de los datos a utilizar, los cuales se encuentran en el archivo MovieReviews.csv. Además, se realiza una revisión de los 5 primeros registros para tener una idea de cómo se encuentran los datos.

In [4]:
df_movies = pd.read_csv('./datos/MovieReviews.csv', encoding='utf-8', index_col=0)

#Se revisa el tamaño del dataset
df_movies.shape

(5000, 2)

In [5]:
#Se revisa una muestra de 5 registros para ver el contenido del dataset
df_movies.sample(5)

,review_es,sentimiento
2809,Decir que esta película apesta sería insultand...,negativo
2290,Vi esto en los teatros y lo adoro absolutament...,positivo
4759,"He visto películas malas antes, ¡pero este se ...",negativo
1841,"Si esta película se examina de cerca, es un po...",positivo
2777,La historia detrás de esta película es muy int...,negativo


### 1.3 Descripción de los datos
En este apartado se realiza una descripción de los datos, para lo cual se realiza una revisión de los tipos de datos.

In [6]:
#Se revisa el tipo de datos de cada columna
df_movies.dtypes

review_es      object
sentimiento    object
dtype: object

Se cuenta con 2 variables de tipo object que representan el comentario y la categorización del comentario.

### 1.4 Análisis de datos

Antes de hacer cualquier cambio al dataframe realizamos una copia de este para no afectar el original.

In [10]:
#Se realiza una copia del dataset original
df_movies_analisis = df_movies.copy()

#### 1.4.1 Análisis de valores nulos
En este apartado se realiza un análisis de los valores nulos, ya que queremos si un comentario es positivo o negativo, por lo que no se puede tener valores nulos en la variable categorización.

In [11]:
#Se revisa la cantidad de valores nulos por columna
df_movies_analisis.isnull().sum()

review_es      0
sentimiento    0
dtype: int64

#### 1.4.2 Análisis de duplicados
En este apartado se realiza un análisis de los duplicados, ya que queremos si un comentario es positivo o negativo, por lo que no se puede tener valores duplicados en la variable comentario.

In [12]:
#Se revisa la cantidad de duplicados por columna
df_movies_analisis.duplicated().sum()

2

Se hace la elimianación de estos datos nulos con el fin tener precisión a la hora de clasiificar los comentarios.

In [13]:
df_movies_analisis.drop_duplicates(inplace=True)
df_movies_analisis.duplicated().sum()

0

## 2. Preparación de datos para el modelo   

Teniendo en cuenta que el objetivo de este proyecto es crear un modelo que por medio de los datos de entrenamiento pueda verificar si un comentario es positivo o negativo, se realiza la preparación de los datos para el modelo.


In [20]:
#Se realiza una copia del dataframe con los cambios del análisis realizado
df_movies_prep = df_movies_analisis.copy()

#### 2.1 Modificación de la columna sentimiento
En este apartado se realiza la modificación de la columna sentimiento, ya que se quiere que esta variable sea binaria, por lo que se realiza la modificación de los valores de la variable sentimiento, para que los valores positivos sean 1 y los negativos sean 0.

In [21]:
#Se modifica la columna 'Sentimiento' para que tenga valores numéricos
df_movies_prep['sentimiento'] = df_movies_prep['sentimiento'].map({'positivo': 1, 'negativo': 0})

#Se visualiza el cambio por medio de head()
df_movies_prep.head()

,review_es,sentimiento
0,Si está buscando una película de guerra típica...,1
1,Supongo que algunos directores de películas de...,1
2,Es difícil contarle más sobre esta película si...,1
3,"La película comienza muy lentamente, con el es...",1
4,Esta película es verdadera acción en su máxima...,1


#### 2.2 Modifiación de letras en minúscula
En este apartado se realiza la modificación de las letras en minúscula, ya que se quiere que todas las palabras estén en minúscula para que no se tengan palabras repetidas.

In [22]:
#Se cambian las letras a minúsculas
df_movies_prep['review_es'] = df_movies_prep['review_es'].str.lower()

#Visualización de los cambios
df_movies_prep.head()

,review_es,sentimiento
0,si está buscando una película de guerra típica...,1
1,supongo que algunos directores de películas de...,1
2,es difícil contarle más sobre esta película si...,1
3,"la película comienza muy lentamente, con el es...",1
4,esta película es verdadera acción en su máxima...,1


#### 2.3 Eliminación de signos de puntuación y caracteres especiales
En este apartado se realiza la eliminación de signos de puntuación y caracteres especiales, ya que no afectan al significado de la palabra.

In [23]:
#Se eliminan todos los caracteres especiales y signos de puntuación
df_movies_prep['review_es'] = df_movies_prep['review_es'].str.replace('[^\w\s]','')

#Se hace '\S' para que no elimine los espacios 
#Visualización de los cambios
df_movies_prep.head()


C:\Users\crist\AppData\Local\Temp\ipykernel_12264\682848269.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies_prep['review_es'] = df_movies_prep['review_es'].str.replace('[^\w\s]','')


,review_es,sentimiento
0,si está buscando una película de guerra típica...,1
1,supongo que algunos directores de películas de...,1
2,es difícil contarle más sobre esta película si...,1
3,la película comienza muy lentamente con el est...,1
4,esta película es verdadera acción en su máxima...,1


#### 2.4 Modificación de palabras con tilde
En este apartado se realiza la modificación de las palabras con tilde, ya que se quiere que todas las palabras estén sin tilde para que no se tengan palabras repetidas.

In [24]:
#Se cambian las tildes por letras sin tilde
df_movies_prep['review_es'] = df_movies_prep['review_es'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

#Visualización de los cambios
df_movies_prep.head()

,review_es,sentimiento
0,si esta buscando una pelicula de guerra tipica...,1
1,supongo que algunos directores de peliculas de...,1
2,es dificil contarle mas sobre esta pelicula si...,1
3,la pelicula comienza muy lentamente con el est...,1
4,esta pelicula es verdadera accion en su maxima...,1


#### 2.5 Eliminación de palabras vacías
En este apartado se realiza la eliminación de las palabras vacías, ya que estas son palabras que no aportan significado a la frase, por lo que no son necesarias para el análisis.
